# CS Capstone Project — Generate Charge Cycle Metrics

**Author:** Jason Waterman  
**Course:** UC Berkeley AI/ML Professional Certificate  
**Created:** 2025-10-18  
**Last Updated:** 2025-10-18  
**Version:** 0.1.0  

---

## 📖 Description
This notebook creates charge cycle metrics for 12V battery beyond the stardard IBS signals from a set of vehicle log files.
It loads raw IBS network logs, extract relevant features, computes metrics, and outputs a file that contains these metrics along with standard IBS signals and SOH, SOH1 targets.
Used for training Decision Tree, Lasso, Ridge, and Random Forest model pipelines.


### 🔑 Objectives
#### Generate charging cycle metrics for
- Voltage response/health
- Charge acceptance/efficiency
- Resistance slope proxies
- Usage/aging context
#### What it computes (the core 12+ extras)
    V_rest — resting voltage before charge (20-min lookback, not-charging majority)
    dV_load_sag — voltage drop after charge ends (proxy for load sag/internal R)
    R_dyn_proxy — ΔV/ΔI across the charge-off edge (if current present)
    V_peak_charge — peak voltage during charge
    time_above_absorption_s — seconds above ~14.2 V (AGM absorption)
    I_bulk_mean — mean current in early charge (first third)
    I_tail_mean — mean current in late charge (last N seconds)
    Ah_in, Wh_in — energy in during the charge
    Ah_out_next, Wh_out_next — energy out until next charge
    eta_coulomb — Ah_out_next / Ah_in
    dSoC_pct — SOC change during the charge
    dSoC_per_Ah — SoC gain per amp-hour (charge efficiency proxy)
    (+ duration_s, V_ripple_pct, SoC_rate_pct_per_min, t80_s, dV_charge)

### 📊 Outputs
- Generate consolidated charge cycle metrics -> save to /outputs/logs/cycle_metrics_all.csv


## 1. Setup and imports

In [1]:
# ===== Standard Library =====
import os, glob
import logging
from pathlib import Path
from datetime import timedelta, datetime
import warnings

# ===== Third-Party Libraries =====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from sklearn.metrics import r2_score, mean_squared_error
from typing import List, Tuple, Optional

# ===== Project Modules =====
# Assuming your capstone repo has a `src/` or `utils/` folder with your modules
from cs_soh_utils import calculate_soh1, detect_cycles_from_voltage, count_cycles_voltage, merge_short_gaps, parse_log_date, compute_log_soh, ensure_dt_index

warnings.filterwarnings("ignore")

In [2]:
# Define file constants
INPUT_CLEANED_CSV = "can_data/cleaned/*.csv"      
OUTPUT_CYCLE_METRICS_CSV = "outputs/logs/cycle_metrics_all.csv"

## 2. Define & Build charge cycle features 

In [3]:
# Which IBS columns to aggregate per cycle (add/remove as needed)
IBS_COLS = [
    "IBS_BatteryVoltage",
    "IBS_BatteryCurrent",
    "IBS_BatteryTemperature",
    "IBS_StateOfCharge",
    "IBS_StateOfHealth",
    "IBS_AvgRi",
    "IBS_AvailableCapacity",
    "IBS_DischargeableAh",
    "IBS_NominalCapacity",
    "IBS_Sulfation",
    "IBS_BatteryDefect",
]

# Which statistics to compute for each IBS column
IBS_STATS = ["mean","median","std","min","max","p10","p90","first","last"]

In [4]:
# ---------- feature helpers ----------
def integrate_ah(t_idx: pd.DatetimeIndex, i_vals) -> float:
    """Trapezoidal integrate current (A) over time to Ah. NaN-safe."""
    if len(t_idx) < 2:
        return 0.0
    i = np.asarray(pd.to_numeric(i_vals, errors="coerce"), dtype=float)
    t = pd.DatetimeIndex(t_idx).sort_values()
    dt_s = np.diff(t.values.astype("datetime64[ns]")) / np.timedelta64(1, "s")
    imid = 0.5 * (i[1:] + i[:-1])
    return float(np.nansum(imid * dt_s) / 3600.0)

def _median_cadence_seconds(idx: pd.DatetimeIndex) -> float:
    if len(idx) < 2:
        return np.nan
    dts = pd.Series(idx).diff().dt.total_seconds().iloc[1:]
    return float(dts.median())

def _largest_step_Rdyn(g: pd.DataFrame, v_col: str, i_col: str, win: int = 3, eps: float = 0.5) -> float:
    """
    Estimate dynamic resistance around the largest |ΔI| step in segment g.
    Works even if g.index has duplicates by using positional indexing.
    """
    if len(g) < 2 or v_col not in g.columns or i_col not in g.columns:
        return np.nan

    # Keep only needed cols; drop rows where both V and I are NaN
    tmp = g[[v_col, i_col]].copy()
    tmp[v_col] = pd.to_numeric(tmp[v_col], errors="coerce")
    tmp[i_col] = pd.to_numeric(tmp[i_col], errors="coerce")
    if tmp[v_col].isna().all() or tmp[i_col].isna().all():
        return np.nan

    # Use positions, not labels (handles duplicate timestamps)
    I = tmp[i_col]
    V = tmp[v_col]
    dI = I.diff()

    # argmax on values to avoid index uniqueness requirements
    try:
        pos = int(np.nanargmax(np.abs(dI.values)))
    except ValueError:
        # all-NaN dI
        return np.nan

    # Build before/after windows with guard rails
    lo = max(0, pos - win)
    hi = min(len(tmp) - 1, pos + win)

    i_before = float(np.nanmedian(I.iloc[lo:pos])) if pos > lo else np.nan
    i_after  = float(np.nanmedian(I.iloc[pos:hi])) if hi > pos else np.nan
    v_before = float(np.nanmedian(V.iloc[lo:pos])) if pos > lo else np.nan
    v_after  = float(np.nanmedian(V.iloc[pos:hi])) if hi > pos else np.nan

    di = i_after - i_before
    if not np.isfinite(di) or abs(di) < eps or not np.isfinite(v_before) or not np.isfinite(v_after):
        return np.nan

    return float((v_before - v_after) / di)


In [5]:
def build_cycle_features(
    dfr: pd.DataFrame,
    vol_col: str = "IBS_BatteryVoltage",
    cur_col: Optional[str] = "IBS_BatteryCurrent",
    soc_col: Optional[str] = "IBS_StateOfCharge",
    gate_col: Optional[str] = "VCU_ChrgSysOperCmd",
    resample: str = "1S",
    median_win_s: int = 3,
    th_on_v: float = 13.4,
    th_off_v: float = 13.2,
    min_len_s: int = 20,          # unified defaults
    merge_gap_s: int = 8,
    absorption_v: float = 14.2,
    rest_min_s: int = 300,        # 5 min rest required before a cycle
    rest_i_thresh: float = 2.0,   # ≤2A counts as rest
    tail_last_s: int = 180,
    t80_target: float = 80.0,
    target_col: Optional[str] = None,
    target_align: str = "end",    # "start" | "mid" | "end"
    target_window_s: int = 30,
    ibs_cols: Optional[list] = None,
    ibs_stats: Optional[list] = None,
    soh_is_fraction: bool = True
) -> pd.DataFrame:
    if not isinstance(dfr.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be a DatetimeIndex.")

    df = dfr.copy()

    # --- IBS aggregation config ---
    if ibs_cols is None:
        ibs_cols = IBS_COLS           # ensure defined in a config cell
    if ibs_stats is None:
        ibs_stats = IBS_STATS         # ensure defined in a config cell
    existing_ibs = [c for c in ibs_cols if c in df.columns]

    # numeric bases
    V_raw = pd.to_numeric(df[vol_col], errors="coerce")
    I_raw = pd.to_numeric(df[cur_col], errors="coerce") if (cur_col and cur_col in df.columns) else None
    SOC_raw = pd.to_numeric(df[soc_col], errors="coerce") if (soc_col and soc_col in df.columns) else None
    gate = pd.to_numeric(df[gate_col], errors="coerce").fillna(0) > 0 if (gate_col and gate_col in df.columns) else None

    # resample/smooth
    V = V_raw.resample(resample).median().ffill()
    if median_win_s and median_win_s > 1:
        V = V.rolling(f"{median_win_s}S", min_periods=1).median()
    I = I_raw.resample(resample).mean().ffill() if I_raw is not None else None
    SOC = SOC_raw.resample(resample).median().ffill() if SOC_raw is not None else None
    if SOC is not None and soh_is_fraction:
        SOC = SOC * 100.0

    # detect & merge
    windows = merge_short_gaps(detect_cycles_from_voltage(V, th_on_v, th_off_v, min_len_s), merge_gap_s)

    # cadence helper
    median_dt = _median_cadence_seconds(V.index)
    to_seconds = (lambda n: float(n) * median_dt) if np.isfinite(median_dt) else (lambda n: np.nan)

    rows = []
    prev_end = None

    for idx, (t0, t1) in enumerate(windows):
        # rest gating
        if rest_min_s > 0 and I_raw is not None:
            t_rest_start = t0 - pd.Timedelta(seconds=rest_min_s)
            pre = df.loc[max(t_rest_start, df.index.min()):t0]
            if len(pre) == 0:
                continue
            rest_like = (pd.to_numeric(pre[cur_col], errors="coerce").abs() <= rest_i_thresh).mean() > 0.8
            if gate is not None:
                rest_like = rest_like and not gate.loc[pre.index].any()
            long_enough = (pre.index[-1] - pre.index[0]).total_seconds() >= (0.8 * rest_min_s)
            if not (rest_like and long_enough):
                continue

        # segments
        seg_o = df.loc[t0:t1]                # original cadence (for integration & IBS stats)
        Vseg  = V.loc[t0:t1]
        Iseg  = I.loc[t0:t1] if I is not None else None
        SOCseg= SOC.loc[t0:t1] if SOC is not None else None

        # core stats
        v_peak = float(Vseg.max())
        v_mean = float(Vseg.mean())
        v_tail_mean = float(Vseg.last(f"{tail_last_s}S").mean()) if len(Vseg) else np.nan
        abs_seconds = to_seconds(int((Vseg >= absorption_v).sum()))
        i_mean = float(Iseg.mean()) if Iseg is not None else np.nan
        i_min  = float(Iseg.min())  if Iseg is not None else np.nan
        soc_start = float(SOCseg.iloc[0]) if (SOCseg is not None and len(SOCseg)) else np.nan
        soc_end   = float(SOCseg.iloc[-1]) if (SOCseg is not None and len(SOCseg)) else np.nan
        soc_gain  = (soc_end - soc_start) if np.isfinite(soc_end) and np.isfinite(soc_start) else np.nan
        if SOCseg is not None and len(SOCseg):
            hit80 = SOCseg[SOCseg >= t80_target]
            t80_s = float((hit80.index[0] - t0).total_seconds()) if len(hit80) else np.nan
        else:
            t80_s = np.nan
        Ah_in = integrate_ah(
            seg_o.index,
            np.clip(pd.to_numeric(seg_o.get(cur_col, np.nan), errors="coerce").values, a_min=0, a_max=None)
        ) if cur_col in seg_o.columns else np.nan
        Ah_out_prev = np.nan
        if prev_end is not None and cur_col in df.columns:
            mid = df.loc[prev_end:t0]
            if len(mid) > 1:
                Ah_out_prev = integrate_ah(
                    mid.index,
                    np.clip(-pd.to_numeric(mid[cur_col], errors="coerce").values, a_min=0, a_max=None)
                )
        R_dyn = _largest_step_Rdyn(seg_o, v_col=vol_col, i_col=cur_col, win=3, eps=0.5) if cur_col in seg_o.columns else np.nan

        # base row
        row = {
            "cycle_idx": idx,
            "cycle_start": t0,
            "cycle_end": t1,
            "duration_s": (t1 - t0).total_seconds(),
            "V_peak_charge": v_peak,
            "V_mean_charge": v_mean,
            "V_tail_mean": v_tail_mean,
            "time_absorption_s": float(abs_seconds) if np.isfinite(abs_seconds) else np.nan,
            "I_bulk_mean": i_mean,
            "I_min": i_min,
            "soc_start_pct": soc_start,
            "soc_end_pct": soc_end,
            "soc_gain_pct": soc_gain,
            "t80_s": t80_s,
            "Ah_in": Ah_in,
            "Ah_out_prev": float(Ah_out_prev) if np.isfinite(Ah_out_prev) else np.nan,
            "R_dyn_ohm": R_dyn,
        }

        # IBS per-cycle aggregates
        if existing_ibs:
            for c in existing_ibs:
                s = pd.to_numeric(seg_o[c], errors="coerce") if c in seg_o.columns else pd.Series(dtype=float)
                if "mean"   in ibs_stats: row[f"{c}__mean"]   = float(np.nanmean(s)) if len(s) else np.nan
                if "median" in ibs_stats: row[f"{c}__median"] = float(np.nanmedian(s)) if len(s) else np.nan
                if "std"    in ibs_stats: row[f"{c}__std"]    = float(np.nanstd(s, ddof=1)) if s.notna().sum() > 1 else np.nan
                if "min"    in ibs_stats: row[f"{c}__min"]    = float(np.nanmin(s)) if s.notna().any() else np.nan
                if "max"    in ibs_stats: row[f"{c}__max"]    = float(np.nanmax(s)) if s.notna().any() else np.nan
                if "p10" in ibs_stats:
                    try:    row[f"{c}__p10"] = float(np.nanpercentile(s.to_numpy(), 10))
                    except: row[f"{c}__p10"] = np.nan
                if "p90" in ibs_stats:
                    try:    row[f"{c}__p90"] = float(np.nanpercentile(s.to_numpy(), 90))
                    except: row[f"{c}__p90"] = np.nan
                if "first" in ibs_stats:
                    s1 = s.dropna()
                    row[f"{c}__first"] = float(s1.iloc[0]) if len(s1) else np.nan
                if "last" in ibs_stats:
                    s1 = s.dropna()
                    row[f"{c}__last"] = float(s1.iloc[-1]) if len(s1) else np.nan

        rows.append(row)
        prev_end = t1

    feats = pd.DataFrame(rows)

    # optional target alignment
    if target_col and target_col in df.columns and not feats.empty:
        target = pd.to_numeric(df[target_col], errors="coerce")
        if soh_is_fraction:
            target = target * 100.0
        half = pd.Timedelta(seconds=target_window_s)
        aligned = []
        for _, r in feats.iterrows():
            t_ref = r["cycle_end"] if target_align == "end" else (r["cycle_start"] if target_align == "start" else r["cycle_start"] + (r["cycle_end"] - r["cycle_start"]) / 2)
            window = target.loc[t_ref - half : t_ref + half]
            aligned.append(float(window.median()) if not window.empty else np.nan)
        feats[f"target_{target_col}"] = aligned

    return feats


In [6]:
# ---------- batch: loop all cleaned CSVs ----------
def _process_one_file(
    path: str,
    *,
    time_col: str | None = "time_utc",
    target_soh1_as_percent: bool = False,   # <--- NEW (optional)
    **kwargs
) -> pd.DataFrame:
    try:
        df = pd.read_csv(path)
    except Exception as e:
        print(f"[SKIP] {path} — read_csv error: {e}")
        return pd.DataFrame()

    try:
        df = ensure_dt_index(df, time_col=time_col, assume_utc=True)
    except Exception as e:
        print(f"[SKIP] {path} — time/index error: {e}")
        return pd.DataFrame()

    # --- NEW: compute file-level SOH1 once for this log ---
    try:
        soh1_val = calculate_soh1(df)              # returns fraction in [0,1]
        if target_soh1_as_percent:
            soh1_val = float(soh1_val) * 100.0     # optional percent format
    except Exception as e:
        print(f"[WARN] {os.path.basename(path)} — calculate_soh1 error: {e}")
        soh1_val = np.nan

    feats = build_cycle_features(df, **kwargs)
    if feats.empty:
        return feats

    # Tag metadata
    feats.insert(0, "source_file", os.path.basename(path))
    if "cycle_idx" in feats.columns:
        feats.insert(1, "cycle_id", feats["source_file"] + "#" + feats["cycle_idx"].astype(str))

    # --- NEW: attach the same file-level target to each cycle row ---
    feats["target_soh1"] = soh1_val

    return feats

def build_cycle_metrics_for_glob(
    input_glob: str,
    output_csv: str | None = None,
    return_report: bool = True,
    **kwargs
) -> pd.DataFrame | tuple[pd.DataFrame, pd.DataFrame]:
    paths = sorted(glob.glob(input_glob))
    if not paths:
        raise FileNotFoundError(f"No files matched: {input_glob}")

    all_feats = []
    report = []
    for p in paths:
        feats = _process_one_file(p, **kwargs)
        if feats is None or feats.empty:
            report.append({"file": os.path.basename(p), "rows": 0, "status": "no cycles"})
        else:
            report.append({"file": os.path.basename(p), "rows": int(len(feats)), "status": "ok"})
            all_feats.append(feats)

    if not all_feats:
        out = pd.DataFrame()
        if output_csv:
            out.to_csv(output_csv, index=False)
            print(f"[WARN] Wrote empty table to: {output_csv}")
        return (out, pd.DataFrame(report)) if return_report else out

    out = pd.concat(all_feats, ignore_index=True)

    # --- NEW: sort by cycle_start (earliest first) ---
    if "cycle_start" in out.columns:
        out = out.sort_values("cycle_start").reset_index(drop=True)

    preferred = [
        "cycle_start", "cycle_end",    # <--- moved to top
        "source_file", "cycle_id", "cycle_idx",
        "duration_s",
        "V_peak_charge", "V_mean_charge", "V_tail_mean",
        "time_absorption_s",
        "I_bulk_mean", "I_min",
        "soc_start_pct", "soc_end_pct", "soc_gain_pct", "t80_s",
        "Ah_in", "Ah_out_prev",
        "R_dyn_ohm",
    ]

    # --- NEW: ensure target_soh1 is the LAST column if present ---
    if "target_soh1" in out.columns:
        cols = [c for c in out.columns if c != "target_soh1"] + ["target_soh1"]
        out = out[cols]

    target_cols = [c for c in out.columns if c.startswith("target_")]
    ordered = [c for c in preferred if c in out.columns] + target_cols + [c for c in out.columns if c not in preferred + target_cols]
    out = out.reindex(columns=ordered)

    if output_csv:
        out.to_csv(output_csv, index=False)
        print(f"[OK] Saved {len(out)} rows to: {output_csv}")

    return (out, pd.DataFrame(report)) if return_report else out



### 3. Run build of charge cycle features 

In [7]:
# ---------- RUN IT ----------

# Run (returns (metrics_df, report_df))
metrics_df, report_df = build_cycle_metrics_for_glob(
    INPUT_CLEANED_CSV,
    output_csv=OUTPUT_CYCLE_METRICS_CSV,
    return_report=True,

    # time handling: set to None if your cleaned CSVs already store time in the index
    time_col="time_utc",

    # unified detection & gating defaults (override here if needed)
    vol_col="IBS_BatteryVoltage",
    cur_col="IBS_BatteryCurrent",
    soc_col="IBS_StateOfCharge",
    gate_col="VCU_ChrgSysOperCmd",
    resample="1S",
    median_win_s=3,

    # Detection (loosen first)
    th_on_v=13.3, th_off_v=13.1,
    min_len_s=10, merge_gap_s=10,

    # Rest gating (turn off to probe)
    rest_min_s=0,           # <-- disable to verify detection works across files
    rest_i_thresh=2.0,
    
    target_soh1_as_percent=False,    # set True if you want % in output

    # Features / target
    absorption_v=14.2,
    target_col=None,        # add later
    soh_is_fraction=True,
)


print("Rows:", 0 if metrics_df is None or metrics_df.empty else len(metrics_df))
print("Report:")
display(report_df)
if metrics_df is not None and not metrics_df.empty:
    print("Files included:", metrics_df['source_file'].nunique())
    display(metrics_df.head(20))


[OK] Saved 75 rows to: outputs/logs/cycle_metrics_all.csv
Rows: 75
Report:


,file,rows,status
0,12V IBS Full load 2025-08-20_22-17-18_L001_IBS...,5,ok
1,12V IBS Full load 2025-08-20_22-17-18_L002_IBS...,4,ok
2,12V IBS Full load 2025-08-20_22-17-18_L003_IBS...,4,ok
3,12V IBS Full load 2025-08-20_22-17-18_L004_IBS...,4,ok
4,12V IBS Full load 2025-08-20_22-17-18_L005_IBS...,5,ok
5,12V IBS Full load 2025-08-20_22-17-18_L006_IBS...,4,ok
6,12V Management 2025-06-25_22-20-47_IBS_only_c...,3,ok
7,12V Management Discharge and Charge Cycle 202...,2,ok
8,12V Management IBS 1hr Reset 2025-06-29_09-32...,1,ok
9,12V_IBS_Charge_Discharge_after_IBS_00007_IBS_o...,9,ok


Files included: 24


,cycle_start,cycle_end,source_file,cycle_id,cycle_idx,duration_s,V_peak_charge,V_mean_charge,V_tail_mean,time_absorption_s,...,IBS_Sulfation__last,IBS_BatteryDefect__mean,IBS_BatteryDefect__median,IBS_BatteryDefect__std,IBS_BatteryDefect__min,IBS_BatteryDefect__max,IBS_BatteryDefect__p10,IBS_BatteryDefect__p90,IBS_BatteryDefect__first,IBS_BatteryDefect__last
0,2025-06-26 02:20:47+00:00,2025-06-26 02:40:03+00:00,12V Management 2025-06-25_22-20-47_IBS_only_c...,12V Management 2025-06-25_22-20-47_IBS_only_c...,0,1156.0,14.115,14.106860,14.100594,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2025-06-26 11:24:45+00:00,2025-06-26 12:02:32+00:00,12V Management 2025-06-25_22-20-47_IBS_only_c...,12V Management 2025-06-25_22-20-47_IBS_only_c...,1,2267.0,14.114,14.102164,14.099900,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2025-06-26 13:09:11+00:00,2025-06-26 13:19:50+00:00,12V Management 2025-06-25_22-20-47_IBS_only_c...,12V Management 2025-06-25_22-20-47_IBS_only_c...,2,639.0,14.114,14.104331,14.100833,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2025-06-26 17:47:23+00:00,2025-06-26 18:22:49+00:00,12V_Management_2025-06-26_13-39-25_IBS_only.csv,12V_Management_2025-06-26_13-39-25_IBS_only.csv#0,0,2126.0,14.113,14.096117,14.101900,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2025-06-26 22:55:27+00:00,2025-06-26 22:55:42+00:00,12V_Management_2025-06-26_13-39-25_IBS_only.csv,12V_Management_2025-06-26_13-39-25_IBS_only.csv#1,1,15.0,14.062,14.044375,14.044375,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2025-06-29 10:07:10+00:00,2025-06-29 10:17:51+00:00,12V Management IBS 1hr Reset 2025-06-29_09-32...,12V Management IBS 1hr Reset 2025-06-29_09-32...,0,641.0,14.115,14.102745,14.099406,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2025-06-29 12:10:35+00:00,2025-06-29 12:11:25+00:00,12V Management Discharge and Charge Cycle 202...,12V Management Discharge and Charge Cycle 202...,0,50.0,14.049,13.989392,13.989392,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2025-06-29 12:43:22+00:00,2025-06-29 13:28:20+00:00,12V Management Discharge and Charge Cycle 202...,12V Management Discharge and Charge Cycle 202...,1,2698.0,14.113,14.068882,14.103272,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2025-08-10 12:34:25+00:00,2025-08-10 12:49:45+00:00,12V_IBS_Charge_Discharge_after_IBS_Reset_00003...,12V_IBS_Charge_Discharge_after_IBS_Reset_00003...,0,920.0,14.118,14.102929,14.109178,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2025-08-10 14:28:13+00:00,2025-08-10 15:05:03+00:00,12V_IBS_Charge_Discharge_after_IBS_Reset_00003...,12V_IBS_Charge_Discharge_after_IBS_Reset_00003...,1,2210.0,14.111,14.027578,14.094683,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
